In [1]:
%%capture
!pip install datasets

In [2]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
from google.colab import drive

In [3]:
drive.mount('sentiment-aware-recommendation')
train_df = pd.read_csv('/content/sentiment-aware-recommendation/MyDrive/sentiment-aware-recommendation/train.csv')
val_df = pd.read_csv('/content/sentiment-aware-recommendation/MyDrive/sentiment-aware-recommendation/val.csv')

Mounted at sentiment-aware-recommendation


In [6]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
print(train_dataset)

Dataset({
    features: ['reviewerID', 'asin', 'reviewText', 'overall', 'sentiment'],
    num_rows: 40000
})


In [11]:
def tokenize_function(examples):
  return tokenizer(examples['reviewText'], padding='max_length', truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [12]:
label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
train_dataset = train_dataset.map(lambda x: {'labels': label_map[x['sentiment']]})
val_dataset = val_dataset.map(lambda x: {'labels': label_map[x['sentiment']]})

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [15]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [21]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir='/content/sentiment-aware-recommendation/MyDrive/sentiment-aware-recommendation/bert_results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/sentiment-aware-recommendation/MyDrive/sentiment-aware-recommendation/bert_logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

In [20]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  f1 = f1_score(labels, preds, average='weighted')
  return {'accuracy': acc, 'f1': f1}

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lijoraju (lijoraju-indian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.398900,0.347348,0.873600,0.854378
2,0.348500,0.360728,0.874000,0.873781
3,0.309800,0.453958,0.874600,0.874541


TrainOutput(global_step=7500, training_loss=0.2745318207105001, metrics={'train_runtime': 3285.5727, 'train_samples_per_second': 36.523, 'train_steps_per_second': 2.283, 'total_flos': 7948185661440000.0, 'train_loss': 0.2745318207105001, 'epoch': 3.0})

In [24]:
model.save_pretrained('/content/sentiment-aware-recommendation/MyDrive/sentiment-aware-recommendation/bert_sentiment_model')
tokenizer.save_pretrained('/content/sentiment-aware-recommendation/MyDrive/sentiment-aware-recommendation/bert_sentiment_model')

('/content/sentiment-aware-recommendation/MyDrive/sentiment-aware-recommendation/bert_sentiment_model/tokenizer_config.json',
 '/content/sentiment-aware-recommendation/MyDrive/sentiment-aware-recommendation/bert_sentiment_model/special_tokens_map.json',
 '/content/sentiment-aware-recommendation/MyDrive/sentiment-aware-recommendation/bert_sentiment_model/vocab.txt',
 '/content/sentiment-aware-recommendation/MyDrive/sentiment-aware-recommendation/bert_sentiment_model/added_tokens.json')